In [1]:
import pandas as pd
import numpy as np
#import gensim
#from gensim.models import Word2Vec
#from gensim.models import FastText
#from sklearn.decomposition import PCA
from matplotlib import pyplot

In [2]:
import matplotlib.pyplot as plt # our main display package
import plotly.graph_objects as go
import string # used for preprocessing
import re # used for preprocessing
import nltk # the Natural Language Toolkit, used for preprocessing
import numpy as np # used for managing NaNs
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords # used for preprocessing
from nltk.stem import WordNetLemmatizer # used for preprocessing
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<div style="border:solid black 2px; padding: 10px">
    
## Load complaints csv file

In [3]:
data = pd.read_csv('complaints.csv')


C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773556 entries, 0 to 2773555
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), object(17)
memory usage: 380.

In [5]:
#taking only first 1,00,000 rows

df =data.iloc[:100000,:]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 100000 non-null  object
 1   Product                       100000 non-null  object
 2   Sub-product                   99998 non-null   object
 3   Issue                         100000 non-null  object
 4   Sub-issue                     92845 non-null   object
 5   Consumer complaint narrative  34520 non-null   object
 6   Company public response       46159 non-null   object
 7   Company                       100000 non-null  object
 8   State                         99574 non-null   object
 9   ZIP code                      99541 non-null   object
 10  Tags                          9721 non-null    object
 11  Consumer consent provided?    69411 non-null   object
 12  Submitted via                 100000 non-null  object
 13  

In [7]:
complain = df[['Product','Consumer complaint narrative']]
complain_df = data[['Product','Consumer complaint narrative']]

In [8]:
complain.head()

,Product,Consumer complaint narrative
0,"Credit reporting, credit repair services, or o...",NaN
1,"Credit reporting, credit repair services, or o...",This consumer reporting agency as defined by l...
2,Debt collection,NaN
3,"Credit reporting, credit repair services, or o...",SOME FRAUDULENT ACCOUNTS WERE ON MY CREDIT REP...
4,Checking or savings account,NaN


In [9]:
# droppping null values

complain.dropna(inplace =True)
complain.reset_index(inplace=True)

complain_df.dropna(inplace =True)
complain_df.reset_index(inplace=True)


C:\Users\user\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [10]:
complain.head()

,index,Product,Consumer complaint narrative
0,1,"Credit reporting, credit repair services, or o...",This consumer reporting agency as defined by l...
1,3,"Credit reporting, credit repair services, or o...",SOME FRAUDULENT ACCOUNTS WERE ON MY CREDIT REP...
2,10,Checking or savings account,I set up a recurring scheduled payment on my c...
3,19,"Credit reporting, credit repair services, or o...",Bank of america is a very reputable bank and h...
4,22,"Credit reporting, credit repair services, or o...",In accordance with the Fair Credit Reporting A...


In [11]:
complain = complain.rename(columns={'Product': 'product','Consumer complaint narrative':'customer_complain'})

In [12]:
complain  = complain[['product','customer_complain']]

In [13]:
complain.head()

,product,customer_complain
0,"Credit reporting, credit repair services, or o...",This consumer reporting agency as defined by l...
1,"Credit reporting, credit repair services, or o...",SOME FRAUDULENT ACCOUNTS WERE ON MY CREDIT REP...
2,Checking or savings account,I set up a recurring scheduled payment on my c...
3,"Credit reporting, credit repair services, or o...",Bank of america is a very reputable bank and h...
4,"Credit reporting, credit repair services, or o...",In accordance with the Fair Credit Reporting A...


In [14]:
complain.shape

(34520, 2)

### Cleaning the text

In [15]:
# function to remove all urls
def remove_urls(text):    
    new_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return new_text

# make all text lowercase
def text_lowercase(text):
    return text.lower()

# remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text

# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text

# lemmatize Words 
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text

#Creating one function so that all functions can be applied at once
def preprocessing(text):
    
    text = text_lowercase(text)
    text = remove_urls(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize(text)
    text = remove_stopwords(text)
    text = lemmatize(text)
    text = ' '.join(text)
    return text

## product mapping : 18 class to 8 class

In [16]:
product_map = {'Vehicle loan or lease': 'vehicle loan',
              'Credit reporting, credit repair services, or other personal consumer reports':'credit report',
              'Credit card or prepaid card': 'card',
              'Money transfer, virtual currency, or money service': 'money transfer',
              'virtual currency': 'money transfer',
              'Mortgage': 'mortgage',
               'Payday loan, title loan, or personal loan': 'loan',
               'Debt collection': 'debt collection',
               'Checking or savings account': 'savings account',
               'Credit card': 'card',
               'Bank account or service': 'savings account',
               'Credit reporting': 'credit report',
               'Prepaid card': 'card',
               'Payday loan': 'loan',
               'Virtual currency': 'money transfer',
               'Student loan': 'loan',
               'Consumer Loan': 'loan',
               'Money transfers': 'money transfer'
              }

complain.replace({"product": product_map}, inplace=True)

In [17]:
complain.head()

,product,customer_complain
0,credit report,This consumer reporting agency as defined by l...
1,credit report,SOME FRAUDULENT ACCOUNTS WERE ON MY CREDIT REP...
2,savings account,I set up a recurring scheduled payment on my c...
3,credit report,Bank of america is a very reputable bank and h...
4,credit report,In accordance with the Fair Credit Reporting A...


In [18]:
count = complain.groupby(["product"]).count()

In [19]:
display(count)

,customer_complain
product,
card,2478
credit report,21432
debt collection,4508
loan,813
money transfer,923
mortgage,1736
savings account,2047
vehicle loan,583


In [20]:
#applying preprocessing function and create new clean_complain column

complain['clean_complain']= complain['customer_complain'].apply(lambda x:preprocessing(x))

In [21]:
complain.head()

,product,customer_complain,clean_complain
0,credit report,This consumer reporting agency as defined by l...,consumer reporting agency defined law violated...
1,credit report,SOME FRAUDULENT ACCOUNTS WERE ON MY CREDIT REP...,fraudulent account credit report inquires neve...
2,savings account,I set up a recurring scheduled payment on my c...,set recurring scheduled payment checking accou...
3,credit report,Bank of america is a very reputable bank and h...,bank america reputable bank long time covid de...
4,credit report,In accordance with the Fair Credit Reporting A...,accordance fair credit reporting act displayin...


In [22]:
#Applying preprocessing and removing '\n' character to customer complain column

#for i in range(complain.shape[0]):
    #complain['customer_complain'][i]=preprocessing(str(complain['customer_complain'][i])) 
#for text in complain.customer_complain:

    #text=text.replace('\n',' ') 

In [23]:
#pip install NLP-python

In [24]:
#pip install en_core_web_lg

In [25]:
#pip install spacy


In [26]:
from NLP import NLP
nlp = NLP()

In [27]:
#!python -m spacy download en_core_web_lg

In [28]:
import en_core_web_lg


In [29]:
nlp = en_core_web_lg.load()

In [30]:
# create functions for vectorization

def get_vec(x) :
    doc = nlp(x)
    vec = doc.vector
    return vec

In [31]:
# create new column vec which have the complain text vectorization

complain['vec'] = complain['clean_complain'].apply(lambda x: get_vec(x))

In [32]:
X = complain['vec']

In [33]:
X = X.values.reshape(-1,1)

In [34]:
X.shape

(34520, 1)

In [35]:
X = np.concatenate(np.concatenate(X,axis =0),axis=0).reshape(-1,300)

In [36]:
X.shape

(34520, 300)

In [37]:
y = complain['product']

In [38]:
X

array([[-0.13681993, -0.45854867,  0.80511445, ..., -1.2215408 ,
        -0.4475213 ,  2.08393   ],
       [ 0.11763671, -1.4172043 ,  0.93198395, ..., -0.5759235 ,
        -0.35577184,  3.0638096 ],
       [-0.5406246 , -0.37838858, -0.8808601 , ..., -0.4415435 ,
        -1.6588451 ,  1.5358824 ],
       ...,
       [ 0.37819836, -1.4845881 ,  0.618526  , ..., -0.8281333 ,
        -0.38920933,  2.1740003 ],
       [-0.04268235, -1.7294381 , -0.7593904 , ..., -0.95458734,
        -0.3333579 ,  2.448873  ],
       [-1.2043273 , -0.87703645, -1.0597833 , ..., -2.3197675 ,
        -0.97081655,  0.91464573]], dtype=float32)

In [39]:
y

0          credit report
1          credit report
2        savings account
3          credit report
4          credit report
              ...       
34515      credit report
34516               card
34517      credit report
34518      credit report
34519    debt collection
Name: product, Length: 34520, dtype: object

In [40]:
# Create the pandas DataFrame
df1 = pd.DataFrame(X)

In [41]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.136820,-0.458549,0.805114,-0.138953,1.414641,0.948406,0.066215,1.460580,-0.587082,-0.848020,...,1.397480,0.905932,-0.072287,0.260658,0.246249,0.064927,0.559086,-1.221541,-0.447521,2.083930
1,0.117637,-1.417204,0.931984,-1.126132,-0.462083,1.851371,-0.399511,0.226660,0.122011,0.549005,...,1.701107,2.033760,-0.263365,0.080754,1.520636,0.689707,0.588681,-0.575924,-0.355772,3.063810
2,-0.540625,-0.378389,-0.880860,-0.675592,2.429070,1.180347,0.631470,1.853935,-1.224610,0.516182,...,1.057666,-0.136302,-0.056258,-0.302484,-0.368588,0.790483,-0.106671,-0.441543,-1.658845,1.535882
3,0.485237,0.328353,-2.014436,0.314919,2.621444,0.785266,1.524495,3.918720,-1.694383,-0.295844,...,1.305003,-2.530739,0.409787,0.019420,-1.317838,0.624033,-0.053283,-0.627893,-2.003273,1.648353
4,-0.083514,-0.626824,1.559217,-0.460597,-0.217662,1.385080,-0.824983,0.354125,0.584702,-0.759060,...,1.335188,1.880489,-0.600188,0.738402,1.776924,-0.154004,0.237224,-0.857443,-0.059611,2.487927


In [42]:
merge =  pd.concat([complain['product'], df1], axis=1)

In [43]:
merge.head()

,product,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,credit report,-0.136820,-0.458549,0.805114,-0.138953,1.414641,0.948406,0.066215,1.460580,-0.587082,...,1.397480,0.905932,-0.072287,0.260658,0.246249,0.064927,0.559086,-1.221541,-0.447521,2.083930
1,credit report,0.117637,-1.417204,0.931984,-1.126132,-0.462083,1.851371,-0.399511,0.226660,0.122011,...,1.701107,2.033760,-0.263365,0.080754,1.520636,0.689707,0.588681,-0.575924,-0.355772,3.063810
2,savings account,-0.540625,-0.378389,-0.880860,-0.675592,2.429070,1.180347,0.631470,1.853935,-1.224610,...,1.057666,-0.136302,-0.056258,-0.302484,-0.368588,0.790483,-0.106671,-0.441543,-1.658845,1.535882
3,credit report,0.485237,0.328353,-2.014436,0.314919,2.621444,0.785266,1.524495,3.918720,-1.694383,...,1.305003,-2.530739,0.409787,0.019420,-1.317838,0.624033,-0.053283,-0.627893,-2.003273,1.648353
4,credit report,-0.083514,-0.626824,1.559217,-0.460597,-0.217662,1.385080,-0.824983,0.354125,0.584702,...,1.335188,1.880489,-0.600188,0.738402,1.776924,-0.154004,0.237224,-0.857443,-0.059611,2.487927


In [44]:
#saving this file to local

merge.to_csv('merge.csv',index = False)

<div style="border:solid black 2px; padding: 10px">
    
## Interacting with data robot

In [45]:
#https://app2.datarobot.com/api/v2
    
import datarobot as dr
dr.Client(endpoint='https://app2.datarobot.com/api/v2', token='NjJhOGE2ZmYwZTU2ZTA1MGZkNzNhODU3OllVZ2VqVnhjT3RvMkUzdEx0UnBrcDhBVnVrM005S25IeS9xUHBpS08zd0U9')


In [46]:
#Create project to datarobot

#Set to the location of your auto-mpg.csv and auto-mpg-test.csv data files
# Example: dataset_file_path = '/Users/myuser/Downloads/auto-mpg.csv'
#training_dataset_file_path = 'merge.csv'
#test_dataset_file_path = 'test.csv'

#Load dataset
#training_dataset = dr.Dataset.create_from_file(training_dataset_file_path)

# Create a new project based on dataset
#project = dr.Project.create_from_dataset(training_dataset.id, project_name='complain3')

In [47]:
#set the target
#project.set_target(target='product')

# Use training data to build models.
#project.wait_for_autopilot()
#model = dr.ModelRecommendation.get(project.id).get_model()



In [48]:
# Get all project list in datarobot for my account

import datarobot as dr
project = dr.Project.list()


In [49]:
project

[Project(complain3),
 Project(complain2),
 Project(complain1),
 Project(Auto complain DR-Client),
 Project(Auto complain DR-Client),
 Project(Auto complain DR-Client),
 Project(Auto MPG DR-Client),
 Project(LendingClub_for_BA_Starter.csv),
 Project(My first ML Project),
 Project(My first ML Project)]

In [50]:
#import datarobot as dr
project = project[0]

In [51]:
#getting project id for project complain 3
project.id

'62d3770488045b8309b02719'

In [52]:
import datarobot as dr
import datarobot as dr
project = dr.Project.get('62d3770488045b8309b02719')
models = project.get_models()
print(models[:5])
model = models[3]

[Model('AVG Blender'), Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)'), Model('Light Gradient Boosted Trees Classifier with Early Stopping (SoftMax Loss) (64 leaves)'), Model('Gradient Boosted Trees Classifier with Early Stopping'), Model('Keras Slim Residual Neural Network Classifier using Training Schedule (1 Layer: 64 Units)')]


In [53]:
model

Model('Gradient Boosted Trees Classifier with Early Stopping')

In [54]:
model.id

'62d37ea98702ef9804a95ddf'

In [55]:
import datarobot as dr

project_id = '62d3770488045b8309b02719'
#model_id = '62d37ea98702ef9804a95dde'
model_id = '62d37ea98702ef9804a95ddf'
project = dr.Project.get(project_id)
model = dr.Model.get(project=project_id,model_id=model_id)


<div style="border:solid black 2px; padding: 10px">
    
## Testing new complain text using data robot trained model

In [56]:
test_complaint_1 = complain_df['Consumer complaint narrative'][100060]

In [57]:
test_complaint_1

'Equifax is ignoring my requests to investigate, verify and remove accounts from my credit report that do not belong to me.'

In [58]:
#Preprocessing data : remove all urls,punctuation,number,removing stopwords,lemmatization

preprocess = preprocessing(test_complaint_1 )

In [59]:
preprocess

'equifax ignoring request investigate verify remove account credit report belong'

In [60]:
#creating vectors for the text to input in the model

vect = get_vec(preprocess)

In [61]:
vect

array([-6.2692001e-02, -1.3132651e+00, -5.9128606e-01,  7.3232017e-02,
        3.4505532e+00,  7.5158906e-01,  1.3345921e-01,  3.0747449e+00,
       -1.6242840e+00, -1.7279819e-01,  3.5665679e+00,  1.5780430e+00,
       -4.8406701e+00,  8.3900899e-01,  1.2495999e-01,  1.2593399e+00,
        2.1151369e+00, -3.9114600e-01, -1.0639498e+00, -2.1781299e-01,
       -1.8734261e+00, -2.2439239e+00, -1.6677971e+00,  3.2376304e-01,
       -2.6712952e+00,  3.7279502e-01, -8.4651792e-01, -1.0288606e+00,
        1.5822400e-01,  2.8487992e-02,  1.9838072e+00,  8.6019838e-01,
       -3.2651401e+00,  1.4347150e+00,  2.2465901e-01, -2.5399399e-01,
       -1.0881530e+00, -8.5787308e-01,  2.9677289e+00,  2.7999721e+00,
       -1.4188271e+00, -4.3959299e-01,  2.5068599e-01,  1.9117720e+00,
       -2.4028060e+00,  2.4526508e-01,  3.5424645e+00, -2.7488332e+00,
       -6.5037197e-01,  9.2753962e-02,  3.9716753e-01,  1.4489430e+00,
        5.2677405e-01, -3.4406810e+00, -2.5478940e+00,  9.8192102e-01,
      

In [62]:
df = pd.DataFrame(vect)

In [63]:
df = df.T

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 300 entries, 0 to 299
dtypes: float32(300)
memory usage: 1.3 KB


In [65]:
df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.062692,-1.313265,-0.591286,0.073232,3.450553,0.751589,0.133459,3.074745,-1.624284,-0.172798,...,0.823387,-0.621792,0.749528,-0.109696,-1.992084,1.143328,0.085498,-1.873049,0.00049,0.951492


In [66]:
#saving to local as dataframe

df.to_csv('df.csv',index = False)

In [67]:

# Using path to local file to generate predictions 
dataset_from_path = project.upload_dataset('df.csv')

predict_job_1 = model.request_predictions(dataset_from_path.id)


In [68]:
predict_job_1 

PredictJob(Model('62d37ea98702ef9804a95ddf'), status=inprogress)

In [69]:
import datarobot as dr
predictions = dr.Predictions.list('62d3770488045b8309b02719')

print(predictions)

[Predictions(prediction_id='62d4182928538abe0f5ccf02', project_id='62d3770488045b8309b02719', model_id='62d37ea98702ef9804a95dde', dataset_id='62d4181128538abe0f5ccef0', includes_prediction_intervals=False, prediction_intervals_size=None, forecast_point=None, predictions_start_date=None, predictions_end_date=None, actual_value_column=None, explanation_algorithm=None, max_explanations=None, shap_warnings=None), Predictions(prediction_id='62d4199c8a1577446f1feade', project_id='62d3770488045b8309b02719', model_id='62d37ea98702ef9804a95dde', dataset_id='62d4197f72643f4b60be593f', includes_prediction_intervals=False, prediction_intervals_size=None, forecast_point=None, predictions_start_date=None, predictions_end_date=None, actual_value_column=None, explanation_algorithm=None, max_explanations=None, shap_warnings=None), Predictions(prediction_id='62d41ce369785fd2bb5cccae', project_id='62d3770488045b8309b02719', model_id='62d37ea98702ef9804a95dde', dataset_id='62d41cc769785fd2bb5ccc91', incl

In [70]:
#get last prediction request id and put it into to prediction_id


import datarobot as dr

preds = dr.Predictions.get("62d3770488045b8309b02719", prediction_id="62dfe9479a6bab8db11979b4")
df = preds.get_all_as_dataframe()

In [71]:
df

,row_id,prediction,class_card,class_credit report,class_debt collection,class_loan,class_money transfer,class_mortgage,class_savings account,class_vehicle loan
0,0,credit report,0.00061,0.992659,0.006033,0.000078,0.000084,0.000091,0.000242,0.000203
